# core

> Python package that helps collect outputs of statistical analyses into tables and export them to LaTex and pdf (similar the ``estout`` and ``esttab`` commands in Stata).

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels import PanelOLS
import estout

In [ ]:
df = pd.DataFrame(np.random.rand(9,2), columns=['y','x'])
df['firmid'] = [1]*3 + [2]*3 + [3]*3
df['time'] = [1,2,3]*3
df['constant'] = 1
df = df.set_index(['firmid','time'])
df

y         x  constant
firmid time                              
1      1     0.617743  0.087632         1
       2     0.789098  0.110738         1
       3     0.979598  0.873752         1
2      1     0.033249  0.289859         1
       2     0.373095  0.295679         1
       3     0.644151  0.077628         1
3      1     0.254426  0.361278         1
       2     0.556160  0.066641         1
       3     0.430909  0.696835         1

In [ ]:
sm1 = sm.OLS(df['y'], df[['constant','x']]).fit()
sm2 = sm.OLS(df['y'], df[['constant','x']]).fit().get_robustcov_results(cov_type='HAC', maxlags=2)
lmres = PanelOLS(df['y'],  df[['constant','x']], entity_effects=True
                 ).fit(cov_type='clustered', cluster_entity=True)

In [ ]:
estout.linearmodels.tstats(lmres)

constant    6.133857
x           0.287433
Name: tstat, dtype: float64

In [ ]:
estout.statsmodels.tstats(sm1)

constant    3.053800
x           0.447218
dtype: float64

In [ ]:
#| export
def extract_stat(res, package: str=None, stat: str=None): 
    return estout.utils.rgetattr(estout,package+'.'+stat)(res)

In [ ]:
extract_stat(lmres, package='linearmodels', stat='tstats')

constant    6.133857
x           0.287433
Name: tstat, dtype: float64

In [ ]:
extract_stat(sm1, package='statsmodels', stat='tstats')

constant    3.053800
x           0.447218
dtype: float64

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()